# Create basic stats table
Given a list of summoner names, we'll create a basic stats table.

In [ ]:
# Set notebook code preview
%config IPCompleter.greedy=True

# Set display width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import Row
from collections import OrderedDict
from pyspark.sql.functions import col, avg

spark = SparkSession.builder.config('spark.driver.extraClassPath', './lib/postgresql-42.2.5.jar').getOrCreate()
url = 'jdbc:postgresql://postgres:5432/lol'
properties = {'user': 'lol', 'password': 'lol'}
sqlc = SQLContext(spark.sparkContext)


In [ ]:
list_of_summoners = ['name_here']

In [ ]:
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

def convert_to_row(d: dict) -> Row:
    return Row(**OrderedDict(sorted(d.items())))

list_of_df = []
list = []
for summoner_name in list_of_summoners:
    get_all_matches_query = f"(select m.* from summoner s join matches map on s.id = map.summoner_id join match m on map.match_id = m.id where s.name = '{summoner_name}') a"
    matches_df = spark.read.jdbc(url=url, table=get_all_matches_query, properties=properties)
    info_df = sqlc.read.json(matches_df.rdd.map(lambda r: r.info))

    for info in info_df.collect():
        game_stat = {}
        game_stat['gameDuration'] = info.gameDuration/60
        game_stat['gameId'] = info.gameId
        game_stat['gameCreation'] = info.gameCreation
        participant_id = 0

        for participant in info.participantIdentities:
            if participant.player.summonerName == summoner_name:
                participant_id = participant.participantId
        # found participantId
        for p in info.participants: 
            if participant_id == p.participantId:
                game_stat['summoner_name'] = summoner_name
                game_stat['goldEarned'] = p.stats.goldEarned
                game_stat['visionScore'] = p.stats.visionScore
                game_stat['totalDamageDealt'] = p.stats.totalDamageDealt
                game_stat['totalDamageTaken'] = p.stats.totalDamageTaken
                game_stat['kills'] = p.stats.kills
                game_stat['deaths'] = p.stats.deaths
                game_stat['assists'] = p.stats.assists
                game_stat['goldPerMinDeltas_10'] = p.timeline.goldPerMinDeltas.__getitem__("0-10") if p.timeline.goldPerMinDeltas else 0            
                game_stat['goldPerMinDeltas_20'] = p.timeline.goldPerMinDeltas.__getitem__('10-20') if p.timeline.goldPerMinDeltas else 0            
                game_stat['goldPerMinDeltas_30'] = p.timeline.goldPerMinDeltas.__getitem__('20-30') if p.timeline.goldPerMinDeltas else 0
                game_stat['creepsPerMinDeltas_10'] = p.timeline.creepsPerMinDeltas.__getitem__('0-10') if p.timeline.goldPerMinDeltas else 0
                game_stat['creepsPerMinDeltas_20'] = p.timeline.creepsPerMinDeltas.__getitem__('10-20') if p.timeline.goldPerMinDeltas else 0
                game_stat['creepsPerMinDeltas_30'] = p.timeline.creepsPerMinDeltas.__getitem__('20-30') if p.timeline.goldPerMinDeltas else 0
                list.append(game_stat)

                                
df = spark.sparkContext.parallelize(list).map(convert_to_row).toDF()           
df.show(100)
df.createOrReplaceTempView('basic_stats')
    

In [ ]:
# new_df = df.select(avg("goldEarned"), avg("visionScore"), avg("totalDamageDealt"), 
#                        avg("totalDamageTaken"), avg("kills"), avg("deaths"), avg("assists"),
#                       avg("goldPerMinDeltas_10"), avg("goldPerMinDeltas_20"), avg("goldPerMinDeltas_30"),
#                       avg("creepsPerMinDeltas_10"), avg("creepsPerMinDeltas_20"), avg("creepsPerMinDeltas_30"))

spark.sql("select summoner_name, avg(kills) from basic_stats group by summoner_name").show()
